# SIGMA

This notebook reproduces the salient characteristics of the [SIGMA](https://ieeexplore.ieee.org/document/9065523) accelerator.

## Imports

Import the necessary modules.

In [ ]:
# HiFiber boilerplate

from fibertree_bootstrap import *

fibertree_bootstrap(style="tree", animation='movie')

# Compilation boilerplate

import os
import sys
sys.path.insert(0, "..")

from src.utils import *

## Initialization

Initialize the input tensors. Tensor shapes and densities can be modified below.

**Warning:** Large tensors will overwhelm the video generation. Either:
1. Use small tensors; as a rule of thumb, fewer than 60 computes (e.g., multiplications) should be required.
2. Do not generate a video; remove the `spacetime` specification from the `mapping` before compiling.

In [ ]:
K = 4
M = 5
N = 6

density = [0.9, 0.5]
seed = 0

A_KM = Tensor.fromRandom(rank_ids=["K", "M"], shape=[K, M], seed=seed, density=density, name="A")
B_KN = Tensor.fromRandom(rank_ids=["K", "N"], shape=[K, N], seed=seed + 1, density=density, name="B")

Note: Small tensors are required for video generation. Partition shapes are decreased for visualization purposes.

## Compile and Run

Below is the TeAAL specification for OuterSpace. To simulate the accelerator:
1. Compile it to HiFiber by running the cell, inserting a new cell
2. Run the new cell, which will
    - Execute the kernel; multiplying the above defined matrices
    - Generate visualizations of the actions of the kernel

Remember, if you are using large tensors, remove the spacetime specification to generate a kernel that does not produce videos. Outputs can still be checked below.

In [ ]:
yaml = """
einsum:
    declaration:
        A: [K, M]
        B: [K, N]
        S: [K, M]
        T: [K, M]
        Z: [M, N]
    expressions:
        - S[k, m] = take(A[k, m], B[k, n], 0)
        - T[k, m] = take(A[k, m], S[k, m], 0)
        - Z[m, n] = T[k, m] * B[k, n]
mapping:
    rank-order:
        A: [K, M]
        B: [K, N]
        S: [K, M]
        T: [K, M]
        Z: [M, N]
    partitioning:
        Z:
            K: [uniform_shape(2)]
            (M, K0): [flatten()]
            MK0: [uniform_occupancy(T.4)]
    loop-order:
        S: [K, M, N]
        T: [K, M]
        Z: [K1, MK01, MK00, N]
    spacetime:
        S:
            space: []
            time: [K, M, N]
        T:
            space: []
            time: [K, M]
        Z:
            space: [MK00]
            time: [K1, MK01, N.coord]
"""

compile(yaml)

## Check Results

Check that generated code computes the correct result.

**Note**: Should be used after compiling and running the kernel (above cell)

In [ ]:
check_matmul(A_KM, B_KN, Z_MN)